In [2]:
%pip install transformers
%pip install torch
%pip install typing
%pip install re

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.0 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 535.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB 8.6 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/12.0 MB 12.1 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/12.0 MB 18.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.8/12.0 MB 22.4 MB/s eta 0:00:01
   ------------ --------------------------- 3.9/12.0 MB 22.6 MB/s eta 0:00:01
   --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/111.0 MB 2.2 MB/s eta 0:00:51
   ---------------------------------------- 0.5/111.0 MB 4.9 MB/s eta 0:00:23
    --------------------------------------- 1.4/111.0 MB 10.0 MB/s eta 0:00:11
   - -------------------------------------- 2.9/111.0 MB 15.6 MB/s eta 0:00:07
   - -------------------------------------- 3.7/111.0 MB 17.1 MB/s eta 0:00:07
   - -------------------------------------- 4.5/111.0 MB 15.8 MB/s eta 0:00:07
   -- ------------------------------------- 6.7/111.0 MB 20.5 MB/s eta 0:00:06
   -- ------------------------------------- 8.3/111.0 MB 22.0 MB/s eta 0:00:05
   --- ------------------------------------ 9.8/111.0 MB 23.1 MB/s eta 0:00:05
   ---- ----------------------------------- 11.2/111.0 MB 29.7 MB/s eta 0:00:04
   ---- ----------------------------------- 12.7/111.0 MB 29.7 MB/s eta 0:00:04
   ----- ---------------------------------- 14.1/111.0 MB 38

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/78.6 kB ? eta -:--:--
     ------------------- ------------------ 41.0/78.6 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 78.6/78.6 kB 621.4 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26421 sha256=364a12fa2a4e8cb18eb099f390cae338c1e6d306f2d0dda04bb550883b3c5b57
  Stored in directory: c:\users\martin\appdata\local\pip\cache\wheels\9d\67\2f\53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing
Note: you may need to restart the kernel to use updated p


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import torch
import re

class SentimentModel():
    def __init__(self, model_name: str = "oliverguhr/german-sentiment-bert"):
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'        
            
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model = self.model.to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\S+', re.MULTILINE)

    def predict_sentiment(self, texts: List[str], output_probabilities = False)-> List[str]:
        texts = [self.clean_text(text) for text in texts]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        # truncation=True limits number of tokens to model's limitations (512)
        encoded = self.tokenizer.batch_encode_plus(texts,padding=True, add_special_tokens=True,truncation=True, return_tensors="pt")
        encoded = encoded.to(self.device)
        with torch.no_grad():
                logits = self.model(**encoded)
        
        label_ids = torch.argmax(logits[0], axis=1)

        if output_probabilities == False:
            return [self.model.config.id2label[label_id.item()] for label_id in label_ids]
        else:
            predictions = torch.softmax(logits[0], dim=-1).tolist()  
            probabilities = []
            for prediction in predictions:
                probabilities += [[[self.model.config.id2label[index], item] for index, item in enumerate(prediction)]]
                
            return [self.model.config.id2label[label_id.item()] for label_id in label_ids], probabilities
        
        
    def replace_numbers(self,text: str) -> str:
        return text.replace("0"," null").replace("1"," eins").replace("2"," zwei")\
            .replace("3"," drei").replace("4"," vier").replace("5"," fünf") \
            .replace("6"," sechs").replace("7"," sieben").replace("8"," acht") \
            .replace("9"," neun")         

    def clean_text(self,text: str)-> str:    
            text = text.replace("\n", " ")        
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)        
            text = self.replace_numbers(text)                
            text = self.clean_chars.sub('', text) # use only text chars                          
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace   
            text = text.strip().lower()
            return text

c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

model = SentimentModel('mdraw/german-news-sentiment-bert')

# Examples from our validation dataset
texts = ['Trump',]

result = model.predict_sentiment(texts)

print(result)

['negative']
